In [3]:
import tensorflow as tf

def image_load(path):
    image = tf.io.read_file(path)
    image_tensor=tf.image.decode_jpeg(image,channels=3)
    image_tensor=tf.image.rgb_to_grayscale(image_tensor)
    image_tensor=tf.cast(image_tensor,tf.float32)
    image_tensor=tf.squeeze(image_tensor)
    image_tensor=image_tensor/255.
    return image_tensor


img = image_load('/tmp/capba/2-3g2vx6rz-0-053-1-1-1.jpg')
print(img.shape)
print(img)


(20, 17)
tf.Tensor(
[[1.         1.         0.98039216 0.9882353  1.         1.
  0.98039216 1.         1.         1.         0.99607843 1.
  1.         1.         0.99215686 0.9843137  1.        ]
 [0.9490196  1.         1.         0.972549   0.96862745 0.9764706
  0.98039216 1.         1.         0.99215686 0.99215686 0.9882353
  0.9882353  0.99215686 1.         1.         1.        ]
 [1.         1.         0.9843137  1.         1.         1.
  0.98039216 1.         1.         1.         1.         1.
  0.99215686 0.9882353  0.99607843 1.         1.        ]
 [1.         0.95686275 0.83137256 0.7921569  0.8980392  0.99607843
  1.         0.98039216 0.98039216 0.99607843 1.         1.
  1.         1.         0.99607843 0.9843137  1.        ]
 [0.9882353  1.         0.8392157  0.6156863  0.65882355 0.8862745
  1.         1.         0.9764706  0.9882353  0.9882353  0.98039216
  0.9882353  1.         1.         0.99607843 1.        ]
 [1.         1.         0.972549   0.827451   0.71372

In [4]:

alllabels = "23456789abcdefghjkmnpqrstuvwxyz"

def label_to_index(label):
    for i in range(len(alllabels)):
        if alllabels[i] == label:
            return float(i)

print(label_to_index('3'))
print(label_to_index('z'))

def index_to_label(idx):
    return alllabels[idx]

print(index_to_label(0))
print(index_to_label(30))

1.0
30.0
2
z


In [8]:

import numpy as np

image_dataset_perelement_numpy=[]
image_dataset_labels=[]
for root, dirs, files in os.walk("/tmp/capba", topdown=False):
    image_count=len(files)
    for name in files:
        filepath = os.path.join(root, name)
        try:
            img = image_load(filepath)
            image_dataset_perelement_numpy.append(np.array(img))
            image_dataset_labels.append(label_to_index(name[0]))
        finally:
            None
            # print('failed', filepath)
        

image_dataset=np.array(image_dataset_perelement_numpy)
all_labels=np.array(image_dataset_labels)

print(image_count)
print(image_dataset.shape, all_labels.shape)

49280
(49280, 20, 17) (49280,)


In [19]:

test_count=int(image_count*0.13)
train_count=image_count-test_count

train_image=image_dataset[test_count:train_count]
train_label=all_labels[test_count:train_count]
test_image=image_dataset[:test_count]
test_label=all_labels[:test_count]

print(train_image.shape, train_label.shape)
print(test_image.shape, test_label.shape)

(36468, 20, 17) (36468,)
(6406, 20, 17) (6406,)


In [30]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, 17)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(31, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(train_image, train_label, epochs=25)

model.evaluate(test_image,  test_label, verbose=2)


Epoch 1/25
1140/1140 [==============================] - 2s 1ms/step - loss: 3.2125 - accuracy: 0.0868
Epoch 2/25
1140/1140 [==============================] - 1s 1ms/step - loss: 2.7436 - accuracy: 0.1813
Epoch 3/25
1140/1140 [==============================] - 1s 1ms/step - loss: 2.3005 - accuracy: 0.2994
Epoch 4/25
1140/1140 [==============================] - 1s 1ms/step - loss: 1.8730 - accuracy: 0.4282
Epoch 5/25
1140/1140 [==============================] - 1s 1ms/step - loss: 1.5491 - accuracy: 0.5203
Epoch 6/25
1140/1140 [==============================] - 1s 1ms/step - loss: 1.3080 - accuracy: 0.5934
Epoch 7/25
1140/1140 [==============================] - 2s 1ms/step - loss: 1.1489 - accuracy: 0.6445
Epoch 8/25
1140/1140 [==============================] - 1s 1ms/step - loss: 1.0312 - accuracy: 0.6791
Epoch 9/25
1140/1140 [==============================] - 1s 1ms/step - loss: 0.9349 - accuracy: 0.7097
Epoch 10/25
1140/1140 [==============================] - 1s 1ms/step - loss: 0.862

[0.3741467297077179, 0.8935372829437256]

In [39]:
model.save('capta-002.h5')


2
z


In [83]:

def topResult(result):
    topIdx = 0
    for i in range(len(result)):
        if result[i] > result[topIdx]:
            topIdx=i
    return index_to_label(topIdx)
    # print(topIdx, index_to_label(topIdx), result[topIdx])
 
procmodel = tf.keras.models.load_model("capta-002.h5")

imgdata=image_load('/tmp/capab/d-5qgxd1bh-2-680,jpg')

result=procmodel.predict(np.array([imgdata,]))

# for v in np.nditer(np.sort(result), order='C'):
#     print(v)
# print(len(np.nditer(result, order='C')))
result = result[0].tolist()
topResult(result)
# for i in range(len(result)):
#     # print(i)
#     print(i, index_to_label(i), result[i])


'd'

In [85]:
for root, dirs, files in os.walk("/tmp/capac", topdown=False):
    for name in files:
        filepath = os.path.join(root, name)
        imgdata = image_load(filepath)
        result=procmodel.predict(np.array([imgdata,]))
        result = result[0].tolist()
        newname = '%s-%s'%(topResult(result), name)
        newpath = os.path.join(root, newname)
        # print(filepath, newpath)
        os.rename(filepath, newpath)




In [87]:
def predict(imgs):
    imgdatas = []
    for img in imgs:
        imgdatas.append(image_load(img))
    result = procmodel.predict(np.array(imgdatas))
    resultStr = ''
    for i in range(len(imgs)):
        resultStr += topResult(result[i].tolist())
    return resultStr

res = predict([
    '/tmp/capac/n-sy57u_up-0-000,jpg',
    '/tmp/capac/p-sy57u_up-1-000,jpg',
    '/tmp/capac/x-sy57u_up-2-000,jpg',
    '/tmp/capac/f-sy57u_up-3-000,jpg',])
print(res)


fnpx
